In [2]:
# Import Required Libraries
import pandas as pd
import numpy as np
from pathlib import Path

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Environment setup complete. Libraries imported and display settings configured.")

Environment setup complete. Libraries imported and display settings configured.


## 1. Define Paths and Locations

In [3]:
# Define project paths
BASE_DIR = Path(r'c:\Users\miray\Desktop\dengue_forecasting_project')
RAW_WEATHER_DIR = BASE_DIR / 'data' / 'raw' / 'weather'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'

# Create processed directory if it doesn't exist
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Define 8 locations in Western Visayas
LOCATIONS = [
    'aklan',
    'antique',
    'bacolodcity',
    'capiz',
    'guimaras',
    'iloilo',
    'iloilocity',
    'negrosocc'
]

print(f"Base Directory: {BASE_DIR}")
print(f"Raw Weather Directory: {RAW_WEATHER_DIR}")
print(f"Processed Data Directory: {PROCESSED_DIR}")
print(f"\nLocations to process: {len(LOCATIONS)}")
for loc in LOCATIONS:
    print(f"  - {loc}")

Base Directory: c:\Users\miray\Desktop\dengue_forecasting_project
Raw Weather Directory: c:\Users\miray\Desktop\dengue_forecasting_project\data\raw\weather
Processed Data Directory: c:\Users\miray\Desktop\dengue_forecasting_project\data\processed

Locations to process: 8
  - aklan
  - antique
  - bacolodcity
  - capiz
  - guimaras
  - iloilo
  - iloilocity
  - negrosocc


## 2. Define Column Mapping

NASA POWER columns → Friendly names for modeling

In [4]:
# Column mapping: NASA POWER -> Friendly Names
COLUMN_MAPPING = {
    'YEAR': 'Year',
    'MO': 'Month',
    'DY': 'Day',
    'PRECTOTCORR': 'Rainfall',           # Precipitation (mm/day)
    'T2M': 'Temp_Avg',                   # Average Temperature (°C)
    'T2M_MAX': 'Temp_Max',               # Maximum Temperature (°C)
    'T2M_MIN': 'Temp_Min',               # Minimum Temperature (°C)
    'RH2M': 'Humidity'                   # Relative Humidity (%)
}

# Missing value code in NASA POWER data
MISSING_VALUE_CODE = -999

print("Column Mapping:")
for old, new in COLUMN_MAPPING.items():
    print(f"  {old:15} -> {new}")

Column Mapping:
  YEAR            -> Year
  MO              -> Month
  DY              -> Day
  PRECTOTCORR     -> Rainfall
  T2M             -> Temp_Avg
  T2M_MAX         -> Temp_Max
  T2M_MIN         -> Temp_Min
  RH2M            -> Humidity


## 3. Function: Load and Clean Single Weather File

In [5]:
def load_weather_csv(filepath, location_name):
    """
    Load a single NASA POWER weather CSV file.
    
    Parameters:
    -----------
    filepath : str or Path
        Path to the weather CSV file
    location_name : str
        Name of the location (e.g., 'aklan')
    
    Returns:
    --------
    pd.DataFrame
        Cleaned daily weather dataframe with datetime index
    """
    from time import perf_counter
    t0 = perf_counter()
    
    # Auto-detect header row by finding '-END HEADER-'
    with open(filepath, 'r') as f:
        header_end_row = 0
        for i, line in enumerate(f):
            if '-END HEADER-' in line:
                header_end_row = i + 1  # Data starts after this line
                break
    
    # Read CSV, skipping detected header rows
    df = pd.read_csv(filepath, skiprows=header_end_row, engine='c', low_memory=False)
    
    # Parse dates from DOY (Day of Year) format
    if 'DOY' in df.columns and 'YEAR' in df.columns:
        df['Date'] = pd.to_datetime(df['YEAR'].astype(str) + '-' + df['DOY'].astype(str), format='%Y-%j')
    elif {'YEAR', 'MO', 'DY'}.issubset(df.columns):
        df['Date'] = pd.to_datetime(df[['YEAR', 'MO', 'DY']].rename(
            columns={'YEAR': 'year', 'MO': 'month', 'DY': 'day'}
        ))
    else:
        raise ValueError(f"Cannot parse date for {location_name}: missing YEAR/DOY or YEAR/MO/DY columns")
    
    # Select and rename relevant columns
    relevant_cols = [col for col in COLUMN_MAPPING.keys() if col in df.columns]
    df_clean = df[['Date'] + relevant_cols].copy()
    
    # Rename columns
    df_clean.rename(columns=COLUMN_MAPPING, inplace=True)
    
    # Replace missing value codes with NaN
    weather_cols = ['Rainfall', 'Temp_Avg', 'Temp_Max', 'Temp_Min', 'Humidity']
    for col in weather_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].replace(MISSING_VALUE_CODE, np.nan)
    
    # Set Date as index
    df_clean.set_index('Date', inplace=True)
    df_clean.sort_index(inplace=True)
    
    # Add location identifier
    df_clean['Location'] = location_name.title()
    
    elapsed = perf_counter() - t0
    print(f"{location_name:12} — rows: {len(df_clean):5} — time: {elapsed:.2f}s")
    
    return df_clean

## 4. Function: Resample Daily to Weekly

In [6]:
def resample_to_weekly(df_daily):
    """
    Resample daily weather data to weekly frequency (DOH epidemiological weeks).
    
    DOH epi weeks start on January 1st each year regardless of day of week,
    then continue in 7-day increments (Jan 1-7, Jan 8-14, etc.)
    
    Aggregation rules:
    - Rainfall: SUM (total weekly rainfall)
    - Temperature & Humidity: MEAN (average weekly values)
    
    Parameters:
    -----------
    df_daily : pd.DataFrame
        Daily weather dataframe with datetime index
    
    Returns:
    --------
    pd.DataFrame
        Weekly aggregated dataframe aligned with DOH epidemiological weeks
    """
    # Create a copy to avoid modifying original
    df_work = df_daily.copy()
    
    # Calculate epidemiological week: days since Jan 1 of each year, divided by 7
    # Week 1 = Jan 1-7, Week 2 = Jan 8-14, etc.
    df_work['year'] = df_work.index.year
    df_work['day_of_year'] = df_work.index.dayofyear
    df_work['epi_week'] = ((df_work['day_of_year'] - 1) // 7) + 1
    
    # Create week start date (Jan 1 + (week-1)*7 days)
    df_work['week_start'] = pd.to_datetime(df_work['year'].astype(str) + '-01-01') + pd.to_timedelta((df_work['epi_week'] - 1) * 7, unit='d')
    
    # Define aggregation rules
    agg_rules = {
        'Rainfall': 'sum',      # Total weekly rainfall
        'Temp_Avg': 'mean',     # Average temperature
        'Temp_Max': 'mean',     # Average of daily maximums
        'Temp_Min': 'mean',     # Average of daily minimums
        'Humidity': 'mean',     # Average humidity
        'Location': 'first'     # Keep location name
    }
    
    # Remove rules for columns that don't exist
    agg_rules = {k: v for k, v in agg_rules.items() if k in df_work.columns}
    
    # Group by week_start date and aggregate
    df_weekly = df_work.groupby('week_start').agg(agg_rules)
    
    # Rename index to 'Date' for consistency
    df_weekly.index.name = 'Date'
    
    # Round numeric columns to 2 decimal places
    numeric_cols = df_weekly.select_dtypes(include=[np.number]).columns
    df_weekly[numeric_cols] = df_weekly[numeric_cols].round(2)
    
    return df_weekly

## 5. Load and Process All Weather Files

In [7]:
# Load all weather files
daily_dfs = []
weekly_dfs = []

for location in LOCATIONS:
    # Construct filepath
    filename = f"weather_{location}.csv"
    filepath = RAW_WEATHER_DIR / filename
    
    if not filepath.exists():
        print(f"WARNING: File not found - {filepath}")
        continue
    
    # Load and clean daily data
    df_daily = load_weather_csv(filepath, location)
    daily_dfs.append(df_daily)
    
    # Resample to weekly
    df_weekly = resample_to_weekly(df_daily)
    weekly_dfs.append(df_weekly)

print(f"\n{'='*60}")
print(f"Successfully processed {len(daily_dfs)} locations")
print(f"{'='*60}")

aklan        — rows:  3630 — time: 0.09s
antique      — rows:  3630 — time: 0.08s
bacolodcity  — rows:  3630 — time: 0.09s
capiz        — rows:  3630 — time: 0.10s
bacolodcity  — rows:  3630 — time: 0.09s
capiz        — rows:  3630 — time: 0.10s
guimaras     — rows:  3630 — time: 0.15s
iloilo       — rows:  3630 — time: 0.12s
guimaras     — rows:  3630 — time: 0.15s
iloilo       — rows:  3630 — time: 0.12s
iloilocity   — rows:  3630 — time: 0.05s
negrosocc    — rows:  3630 — time: 0.07s

Successfully processed 8 locations
iloilocity   — rows:  3630 — time: 0.05s
negrosocc    — rows:  3630 — time: 0.07s

Successfully processed 8 locations


## 6. Combine into Regional Dataframes

In [8]:
# Combine all daily dataframes
df_daily_combined = pd.concat(daily_dfs, axis=0)
df_daily_combined.sort_index(inplace=True)

print("Daily Combined Dataset:")
print(f"  Shape: {df_daily_combined.shape}")
print(f"  Date range: {df_daily_combined.index.min()} to {df_daily_combined.index.max()}")
print(f"  Locations: {df_daily_combined['Location'].unique()}")
print(f"\nFirst few rows:")
print(df_daily_combined.head(10))

print("\n" + "="*60)

# Combine all weekly dataframes
df_weekly_combined = pd.concat(weekly_dfs, axis=0)
df_weekly_combined.sort_index(inplace=True)

print("\nWeekly Combined Dataset:")
print(f"  Shape: {df_weekly_combined.shape}")
print(f"  Date range: {df_weekly_combined.index.min()} to {df_weekly_combined.index.max()}")
print(f"  Locations: {df_weekly_combined['Location'].unique()}")
print(f"\nFirst few rows:")
print(df_weekly_combined.head(10))

Daily Combined Dataset:
  Shape: (29040, 7)
  Date range: 2016-01-01 00:00:00 to 2025-12-08 00:00:00
  Locations: ['Aklan' 'Iloilo' 'Guimaras' 'Bacolodcity' 'Antique' 'Negrosocc'
 'Iloilocity' 'Capiz']

First few rows:
            Year  Rainfall  Temp_Avg  Temp_Max  Temp_Min  Humidity  \
Date                                                                 
2016-01-01  2016      1.81     26.12     28.91     24.29     86.50   
2016-01-01  2016      0.99     25.85     29.70     23.36     84.11   
2016-01-01  2016      0.31     27.32     29.99     25.62     80.90   
2016-01-01  2016      0.40     25.07     28.96     22.79     84.54   
2016-01-01  2016      0.72     27.83     29.95     26.50     79.06   
2016-01-01  2016      0.13     25.40     28.70     23.42     83.32   
2016-01-01  2016      0.31     27.32     29.99     25.62     80.90   
2016-01-01  2016      1.81     26.12     28.91     24.29     86.50   
2016-01-02  2016      0.05     25.33     29.54     21.96     83.32   
2016-01-02 

## 7. Pivot to Wide Format (Multi-Output LSTM Preparation)

For Multi-Output LSTM modeling, we need:
- Each row = 1 week
- Columns = weather features for ALL 8 locations
- Format: [Aklan_Rainfall, Aklan_Temp_Avg, ..., Negrosocc_Rainfall, Negrosocc_Temp_Avg, ...]

This allows the LSTM to:
1. Learn shared regional weather patterns
2. Capture location-specific variations
3. Output 8 separate dengue forecasts (one per province)

In [9]:
# Pivot weekly data to wide format for multi-output LSTM
# Reset index to make Date a column for pivoting
df_pivot_prep = df_weekly_combined.reset_index()

# Get weather feature columns (excluding Location)
weather_features = [col for col in df_pivot_prep.columns if col not in ['Date', 'Location']]

print(f"Weather features to pivot: {weather_features}")
print(f"Locations: {df_pivot_prep['Location'].unique()}\n")

# Create wide format dataframe
df_wide_list = []

for feature in weather_features:
    # Pivot each feature: rows=dates, columns=locations
    pivot_df = df_pivot_prep.pivot(index='Date', columns='Location', values=feature)
    
    # Rename columns to Location_Feature format (e.g., 'Aklan_Rainfall')
    pivot_df.columns = [f"{col}_{feature}" for col in pivot_df.columns]
    
    df_wide_list.append(pivot_df)

# Concatenate all pivoted features horizontally
df_weather_wide = pd.concat(df_wide_list, axis=1)

# Sort columns alphabetically for consistency
df_weather_wide = df_weather_wide[sorted(df_weather_wide.columns)]

print("Wide Format Weather Dataset (Multi-Output LSTM Ready):")
print(f"  Shape: {df_weather_wide.shape}")
print(f"  Date range: {df_weather_wide.index.min()} to {df_weather_wide.index.max()}")
print(f"  Total weeks: {len(df_weather_wide)}")
print(f"  Total features (8 locations × {len(weather_features)} variables): {df_weather_wide.shape[1]}")
print(f"\nColumn structure:")
for i, col in enumerate(df_weather_wide.columns[:5]):
    print(f"  {col}")
print("  ...")
for i, col in enumerate(df_weather_wide.columns[-3:]):
    print(f"  {col}")
print(f"\nFirst few rows:")
print(df_weather_wide.head())
print(f"\nLast few rows:")
print(df_weather_wide.tail())

Weather features to pivot: ['Rainfall', 'Temp_Avg', 'Temp_Max', 'Temp_Min', 'Humidity']
Locations: ['Aklan' 'Iloilocity' 'Iloilo' 'Guimaras' 'Capiz' 'Bacolodcity' 'Antique'
 'Negrosocc']

Wide Format Weather Dataset (Multi-Output LSTM Ready):
  Shape: (526, 40)
  Date range: 2016-01-01 00:00:00 to 2025-12-03 00:00:00
  Total weeks: 526
  Total features (8 locations × 5 variables): 40

Column structure:
  Aklan_Humidity
  Aklan_Rainfall
  Aklan_Temp_Avg
  Aklan_Temp_Max
  Aklan_Temp_Min
  ...
  Negrosocc_Temp_Avg
  Negrosocc_Temp_Max
  Negrosocc_Temp_Min

First few rows:
            Aklan_Humidity  Aklan_Rainfall  Aklan_Temp_Avg  Aklan_Temp_Max  \
Date                                                                         
2016-01-01           82.87            3.22           25.81           28.96   
2016-01-08           81.81            2.32           26.61           30.03   
2016-01-15           77.71            4.20           26.07           29.97   
2016-01-22           82.18       

## 8. Check for Missing Values

In [10]:
print("Missing Values in Wide Format Dataset:")
print("="*60)
missing_summary = df_weather_wide.isna().sum()
missing_pct = (df_weather_wide.isna().sum() / len(df_weather_wide) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing_Count': missing_summary,
    'Missing_Percent': missing_pct
})

# Show only columns with missing values
missing_df_filtered = missing_df[missing_df['Missing_Count'] > 0]

if len(missing_df_filtered) > 0:
    print(f"\nColumns with missing values ({len(missing_df_filtered)} / {len(missing_df)}):")
    print(missing_df_filtered)
    print("\n⚠️  Missing values detected. Will need interpolation in next step.")
else:
    print("\n✅ No missing values detected!")

print(f"\nTotal missing cells: {missing_summary.sum()}")
print(f"Overall completeness: {((1 - missing_summary.sum() / df_weather_wide.size) * 100):.2f}%")

Missing Values in Wide Format Dataset:

✅ No missing values detected!

Total missing cells: 0
Overall completeness: 100.00%


## 9. Save Processed Data

In [11]:
# 1. Save long format (location as column) - useful for EDA and visualizations
output_weekly_long = PROCESSED_DIR / 'weather_weekly_long.csv'
df_weekly_combined.to_csv(output_weekly_long)
print(f"✅ Saved: {output_weekly_long}")
print(f"   Shape: {df_weekly_combined.shape}")
print(f"   Format: Long (stacked by location)")

# 2. Save wide format (location-feature columns) - LSTM MODEL INPUT
output_weekly_wide = PROCESSED_DIR / 'weather_weekly_wide.csv'
df_weather_wide.to_csv(output_weekly_wide)
print(f"\n✅ Saved: {output_weekly_wide}")
print(f"   Shape: {df_weather_wide.shape}")
print(f"   Format: Wide (8 locations × {len(weather_features)} features = {df_weather_wide.shape[1]} columns)")
print(f"   Purpose: Multi-Output LSTM training input")

# 3. Optional: Save daily combined data (for future granular analysis)
output_daily = PROCESSED_DIR / 'weather_daily_combined.csv'
df_daily_combined.to_csv(output_daily)
print(f"\n✅ Saved: {output_daily}")
print(f"   Shape: {df_daily_combined.shape}")
print(f"   Format: Daily frequency (optional reference)")

✅ Saved: c:\Users\miray\Desktop\dengue_forecasting_project\data\processed\weather_weekly_long.csv
   Shape: (4208, 6)
   Format: Long (stacked by location)



✅ Saved: c:\Users\miray\Desktop\dengue_forecasting_project\data\processed\weather_weekly_wide.csv
   Shape: (526, 40)
   Format: Wide (8 locations × 5 features = 40 columns)
   Purpose: Multi-Output LSTM training input

✅ Saved: c:\Users\miray\Desktop\dengue_forecasting_project\data\processed\weather_daily_combined.csv
   Shape: (29040, 7)
   Format: Daily frequency (optional reference)

✅ Saved: c:\Users\miray\Desktop\dengue_forecasting_project\data\processed\weather_daily_combined.csv
   Shape: (29040, 7)
   Format: Daily frequency (optional reference)
